In [2]:
import requests
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [5]:
GEO_LOCATION_API_KEY = os.getenv('GEO_LOCATION_API_KEY')

In [34]:
request_url = f'https://api.tomtom.com/search/2/poiSearch/restaurant.json?key={GEO_LOCATION_API_KEY}&limit=5&lat=-23.576604&lon=-46.668532&radius=1000'

In [35]:
response = requests.get(request_url)

In [36]:
response.content

b'{"summary":{"query":"restaurant","queryType":"NON_NEAR","queryTime":31,"numResults":5,"offset":0,"totalResults":380,"fuzzyLevel":1,"geoBias":{"lat":-23.576604,"lon":-46.668532}},"results":[{"type":"POI","id":"076009026588247","score":2.57452631,"dist":67.05862458814353,"info":"search:ta:076009026588247-BR","poi":{"name":"E Sports Burguer","categorySet":[{"id":7315}],"categories":["restaurant"],"classifications":[{"code":"RESTAURANT","names":[{"nameLocale":"en-US","name":"restaurant"}]}]},"address":{"streetNumber":"602","streetName":"Rua General Fonseca Teles","municipalitySubdivision":"Jardim Paulista","municipality":"S\xc3\xa3o Paulo","countrySubdivision":"S\xc3\xa3o Paulo","postalCode":"01433","extendedPostalCode":"01433-020","countryCode":"BR","country":"Brasil","countryCodeISO3":"BRA","freeformAddress":"Rua General Fonseca Teles, 602, 01433-020, S\xc3\xa3o Paulo","localName":"S\xc3\xa3o Paulo"},"position":{"lat":-23.57711,"lon":-46.66889},"viewport":{"topLeftPoint":{"lat":-23.576